<a href="https://colab.research.google.com/github/TimothyJan/RNN_Text_Generation/blob/main/Project_4_RNN_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spring 2022 - CPSC 585-section-13883
Project 4 - Recurrent Neural Networks Text Generation
Sean Javiya
Timothy Jan
Timothy Kheang

In Project 4, we use an RNN-based language model to generate text for a creative application.

Goals for this project are:
<ul>
  <li>Reading about how RNNs can be used to generate text, and examining several different applications after training on different text corpora.</li>
  <li>Using a multi-layer RNN to train and sample from a character-level language model.</li>
  <li>Adapting and reusing published model code.</li>
  <li>Having some fun.</li>
</ul>

In [28]:
import tensorflow as tf
import numpy as np
import os
import time
import urllib.request, json 

Download the Elder Scroll's dataset

In [3]:
# imported the elder scrolls json file
with urllib.request.urlopen("https://raw.githubusercontent.com/hmi-utwente/video-game-text-corpora/master/The%20Elder%20Scrolls/data/imperial_library_20200626.json") as url:
    data_dict = json.loads(url.read().decode())

Read the Data

In [4]:
text=""
for key in data_dict:
  text += data_dict[key]['description'] +' ' + data_dict[key]['title'] + ' ' + data_dict[key]['text']
text[:100]

"The  last words of a world-renowned archaeologist. A Dying Man's Last Words It's been many days sinc"

In [5]:
# unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

118 unique characters


<h1>Process the text</h1>
<h2>Vectorize the text.</h2>
Before training, you need to convert the strings to a numerical representation.
The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Create the tf.keras.layers.StringLookup layer. It onverts from tokens to character IDs:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[65, 66, 67, 68, 69, 70, 71], [88, 89, 90]]>

This layer recovers the characters from the vectors of IDs, and returns them as a tf.RaggedTensor of characters:

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Use tf.strings.reduce_join to join the characters back into strings.

In [9]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [10]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


<h2>The Prediction Task</h2>
Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

<h2>Create training examples and targets</h2>
Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [11]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(9689501,), dtype=int64, numpy=array([53, 72, 69, ..., 80, 69, 16])>

In [12]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
h
e
 
 
l
a
s
t
 


The batch method lets you easily convert these individual characters to sequences of the desired size.

In [13]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'T' b'h' b'e' b' ' b' ' b'l' b'a' b's' b't' b' ' b'w' b'o' b'r' b'd'
 b's' b' ' b'o' b'f' b' ' b'a' b' ' b'w' b'o' b'r' b'l' b'd' b'-' b'r'
 b'e' b'n' b'o' b'w' b'n' b'e' b'd' b' ' b'a' b'r' b'c' b'h' b'a' b'e'
 b'o' b'l' b'o' b'g' b'i' b's' b't' b'.' b' ' b'A' b' ' b'D' b'y' b'i'
 b'n' b'g' b' ' b'M' b'a' b'n' b"'" b's' b' ' b'L' b'a' b's' b't' b' '
 b'W' b'o' b'r' b'd' b's' b' ' b'I' b't' b"'" b's' b' ' b'b' b'e' b'e'
 b'n' b' ' b'm' b'a' b'n' b'y' b' ' b'd' b'a' b'y' b's' b' ' b's' b'i'
 b'n' b'c' b'e'], shape=(101,), dtype=string)


In [14]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"The  last words of a world-renowned archaeologist. A Dying Man's Last Words It's been many days since"
b' the collapse. I have had many good and  exciting adventures. I fear this is the last. I am still uns'
b"ure what  happened. Was it a trap that caused the collapse? I didn't hear the  click of any device. P"
b'erhaps it was simply a freak accident, and I was  simply in the wrong place at the right time. Regard'
b'less, I now lie here,  half buried in the collapse, with crushed legs. The pain was unbearable  for t'


For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [15]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"The  last words of a world-renowned archaeologist. A Dying Man's Last Words It's been many days sinc"
Target: b"he  last words of a world-renowned archaeologist. A Dying Man's Last Words It's been many days since"


<h2>Create training batches</h2>
You used tf.data to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

<h1>Build The Model</h1>
This section defines the model as a keras.Model subclass (For details see Making new Layers and Models via subclassing).

This model has three layers:
<ul>
  <li>tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;</li>
  <li>tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)</li>
  <li>tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.</li>
</ul>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

<h1>Try the model</h1>
Now run the model to see that it behaves as expected.

First check the shape of the output:

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 119) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  30464     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  121975    
                                                                 
Total params: 4,090,743
Trainable params: 4,090,743
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.
using the first example in the batch, this gives us, at each timestep, a prediction of the next character index:

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 63,  86,  72,  63, 109,  12,  86,  21, 118,  46,  89,  47,   5,
       110,  45,  41,   9,  83,  12,  42,  38,  33,  94,  46,  91,  31,
        66,  48,  63, 111,  61,   5,  13,  80,  79,  47,  55,  42, 116,
         9,  91,  61,  20,  27,  74,  45,  77,  27, 103,   7,  26, 108,
        39,  46,  68,  30,  49,  78,  36,   1,  72,  22,  96,  68,  49,
       111,  81, 100,  77,  74,   8,  49,  91,  61,  27,  98,  36,  61,
        46,  74,  69,  99, 104,  96,  43,  52, 116,  96, 105, 109,  41,
        99,  43,   7,  85,  91,  78,  81, 104,  74])

Decoded text predicted by untrained model:


In [22]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' for his use; so that the poor Netch died, not from his wounds, but from the greed and carelessness '

Next Char Predictions:
 b'_vh_\xe2\x80\x94*v3\xe2\x82\xacMyN"\xe2\x80\x98LH\'s*IE?\xc2\xb8M|=bO_\xe2\x80\x99\\"+poNVI\xe2\x80\xb9\'|\\29jLm9\xc3\xae%8\xe2\x80\x93FMd<PnC\th4\xc3\x9fdP\xe2\x80\x99q\xc3\xa8mj&P|\\9\xc3\xa2C\\Mje\xc3\xa4\xc3\xaf\xc3\x9fJS\xe2\x80\xb9\xc3\x9f\xc3\xb4\xe2\x80\x94H\xc3\xa4J%u|nq\xc3\xafj'


<h1>Train the model</h1>
Given the previous RNN state, and the input this time step, predict the class of the next character.
<h2>Attach an optimizer, and a loss function</h2>
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, you need to set the from_logits flag.

In [24]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 119)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.778763, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [25]:
tf.exp(example_batch_mean_loss).numpy()

118.957085

Configure the training procedure using the tf.keras.Model.compile method. Use tf.keras.optimizers.Adam with default arguments and the loss function.

In [26]:
model.compile(optimizer='adam', loss=loss)

<h2>Configure Checkpoints</h2>
Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

<h2>Execute the training</h2>
To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [31]:
EPOCHS = 1
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

   2/1498 [..............................] - ETA: 2:06:12 - loss: 3.7761

KeyboardInterrupt: ignored